In [3]:
import re
import pandas as pd

In [4]:
fileLocation = "data1.csv"

In [5]:
data = pd.read_csv(fileLocation)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc0 in position 0: invalid start byte